In [ ]:
import CaboCha
from pyknp import KNP
import re
import collections
import os
import hashlib

def make_answer(sentence):

    When = []
    Where = []
    What = []
    Why = []
    Who = []
    How = []
    w = []
    c = CaboCha.Parser('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

    words = []
    poss = []
    chunkId = [] 
    toId = []
    ID = -1
    scores = []
    howId = []



    nani = ['は','が','を','の','に','と'] #Whatの後に続く助詞
    naze =['だから', 'それで', 'そのため', 'このため', 'そこで', 'したがって', 'ゆえに', 'それゆえに','から','んで','ため','ので']#Whyの後に続く接続助詞


    tree =  c.parse(sentence) 

    result = knp.parse(sentence)

    
    answers = {"who_text":None, "who_start":-1,"what_text":None,"what_start":-1,"where_text":None,"where_start":-1,
                                    "when_text":None,"when_start":-1,"why_text":None,"why_start":-1,"how_text":None,"how_start":-1}

    for i in range(0, tree.size()):
        token = tree.token(i)
        #単語を取得
        word = token.surface
        words.append(word) #文字をスタック
        #品詞を取得
        pos = token.feature.split(",")
        poss.append(pos) #品詞をスタック

        #チャンクID, かかり先ID, スコアを取得
        if token.chunk != None:
                ID += 1
                ID2 = token.chunk.link
                score  = token.chunk.score
        chunkId.append(ID)
        scores.append(score)
        toId.append(ID2)



    num = len(words)-1
    while num > -1:

    ##Why##
        for c in naze:  
            if words[num] == c: #続く助詞がWhyの後に続くものの場合
       
                w.insert(0,words[num])
                nowId = chunkId[num] #最初のチャンクIDを保存

                while num>0:

                    if chunkId[num-1] != nowId and toId[num-1] > nowId and words[num-1] != '(': #今のチャンクIdが最初のIdでなく, かかり先Idが最初のId出ない
                        break
                    num -=1
                    w.insert(0,words[num])
                    answers["why_start"]= num
                Why.insert(0,''.join(w))
                w = []


        if words[num] == 'に' and words[num-1] == 'ため':
            w.insert(0,words[num])
            nowId = chunkId[num] #最初のチャンクIDを保存
            num -=1
            while num>-1:

                if chunkId[num-1] != nowId and toId[num-1] > nowId and words[num-1] != '(': #今のチャンクIdが最初のIdでなく, かかり先Idが最初のId出ない
                    break
                w.insert(0,words[num])   
                answers["why_start"]= num
                num -=1
            Why.insert(0,''.join(w))
            w = []        
    ####    


    ##How##    
        elif toId[num] == -1 or words[num] == '。':
            w.insert(0, words[num])
            howId.append(chunkId[num])
            num -= 1
            while num > -1:
                if poss[num][0] =='助詞'and poss[num-1][0] != '動詞':
                    break
                w.insert(0,words[num])
                answers["how_start"]= num
                howId.append(chunkId[num])
                num -=1

            How.insert(0, ''.join(w))
            w=[]
    ####


    ##What## 

        elif toId[num] in howId and (poss[num][1] == '係助詞' or poss[num][1] == '格助詞' or (poss[num][1] == '接続助詞')) and (words[num] in nani): 

            w.insert(0,words[num])
            whatId = chunkId[num]
            num -= 1
     #       while num > -1:
    #            if chunkId[num] != whatId:
    #
     #               if ((toId[num] != whatId) and (toId[num] in howId == False)) or (poss[num][0] == '助詞' and (poss[num][1] != '係助詞' and poss[num][1] != '格助詞' and poss[num][1] != '終助詞')):
     #                   break
        #            elif scores[num] > 1.5:
     #           w.insert(0, words[num])
       #         num -= 1

            while chunkId[num] == chunkId[num-1]:
                w.insert(0, words[num])
                num -= 1

            w.insert(0, words[num])   
            answers["what_start"]= num
            What.insert(0,''.join(w))
            w = []
    ####    
        else:
            num -= 1
    answers["who_text"] = Who;
    answers["what_text"] = What;
    answers["why_text"] = Why;
    answers["how_text"] = How;

    return(answers)


In [ ]:
import random
import csv
import re


S = []
with open('./text/livedoor.tsv') as f1:
    l=f1.readlines()
    leng = len(l)
    L=random.sample(l,leng)
    for row in L:
        row_l = row.split('\t')
        tmp = re.sub('\u3000', "", row_l[1].split(' ')[0])
        tmp = re.sub('!!', "", tmp)
        tmp = re.sub('!?', "", tmp)
        S.append(tmp)

In [ ]:
dataset = []
for sen in S[0:100]:

    sentence = sen


    buf = ''
    while len(buf) < 20:
        buf += hashlib.md5(os.urandom(100)).hexdigest()
    answers = make_answer(sentence)
    paragraph = {"text":sentence,"ID":buf,"answers":answers}
    dataset.append(paragraph)